# Solved Assignment

-by Shubham Chillal

***

### Importing Libraries:

In [1]:
import requests
from zipfile import ZipFile
import numpy as np
import pandas as pd
import datetime as dt
from forex_python.bitcoin import BtcConverter

### Saving the links for the data:

In [2]:
m1=requests.get("https://data.binance.vision/data/spot/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-01.zip")
m2=requests.get("https://data.binance.vision/data/spot/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-02.zip")
m3=requests.get("https://data.binance.vision/data/spot/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-03.zip")
m4=requests.get("https://data.binance.vision/data/spot/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-04.zip")
links=[m1,m2,m3,m4]

***

### Assignment I - Basic Python - 3 points

The purpose of this assignment is to assess your ability to use Python to retrieve a large public dataset and store it in memory using Pandas for further analysis. The documentation and endpoint is laid out below this section. You are expected to retrieve data at the tick level for BTCUSDT for the first 3 days of March.

In [3]:
# Downloading and extracting the zip files:

csv_files=[]
for link in links:
    with open("data","wb") as f:
        f.write(link.content)
    with ZipFile("data","r")as data_zip:
        data_zip.extractall()
    csv_files.append(data_zip.namelist()) #Storing name of csv files to further access them

csv_files    

[['BTCUSDT-trades-2022-03-01.csv'],
 ['BTCUSDT-trades-2022-03-02.csv'],
 ['BTCUSDT-trades-2022-03-03.csv'],
 ['BTCUSDT-trades-2022-03-04.csv']]

In [4]:
# Combinning all the data and mentioning column names:
column_names=["trade Id","Traded_Price","Traded_Size","Traded_Notional","Timestamp","isBuyerMaker","isBestMatch"]

df_li=[]
for file in csv_files:
    df=pd.read_csv(file[0],names=column_names)
    df_li.append(df)
    
df=pd.concat(df_li,axis=0,ignore_index=True)
df.head(3)

,trade Id,Traded_Price,Traded_Size,Traded_Notional,Timestamp,isBuyerMaker,isBestMatch
0,1274557543,43160.00,0.00056,24.169600,1646092800000,True,True
1,1274557544,43160.01,0.00052,22.443205,1646092800003,False,True
2,1274557545,43160.01,0.00182,78.551218,1646092800004,False,True


In [5]:
#Converting Timestamp into meaningfull format:
df.Timestamp=[dt.datetime.fromtimestamp(x/1000.0) for x in df.Timestamp]
df.head(3)

,trade Id,Traded_Price,Traded_Size,Traded_Notional,Timestamp,isBuyerMaker,isBestMatch
0,1274557543,43160.00,0.00056,24.169600,2022-02-28 13:00:00.000,True,True
1,1274557544,43160.01,0.00052,22.443205,2022-02-28 13:00:00.003,False,True
2,1274557545,43160.01,0.00182,78.551218,2022-02-28 13:00:00.004,False,True


In [6]:
# As mentioned in the question, slicing the data of first three days of march:
# And including only needed columns:

March_01_03=df[["Traded_Price","Traded_Size","Traded_Notional","Timestamp"]][(df.Timestamp>'2022-02-28 23:59:59.999000')&(df.Timestamp<'2022-03-04 00:00:00.00')].reset_index(drop=True)
print("-"*60,"\n","Start time of Data :",March_01_03.Timestamp.min(),"\n","End time of Data :",March_01_03.Timestamp.max(),"\n","-"*60)

March_01_03.head()

------------------------------------------------------------ 
 Start time of Data : 2022-03-01 00:00:00 
 End time of Data : 2022-03-03 23:59:59.999000 
 ------------------------------------------------------------


,Traded_Price,Traded_Size,Traded_Notional,Timestamp
0,43578.88,0.00106,46.193613,2022-03-01 00:00:00.000
1,43578.88,0.00065,28.326272,2022-03-01 00:00:00.002
2,43578.87,0.00431,187.824930,2022-03-01 00:00:00.004
3,43578.87,0.00036,15.688393,2022-03-01 00:00:00.005
4,43578.87,0.00063,27.454688,2022-03-01 00:00:00.005


***

### Assignment II - Simple Forward Looking Market Making Strategy - 5 points 
You may use any Python library you are familiar with to do this, but pandas and backtrader are preferred. Using the above dataset, you are to create a Market Making strategy in Python that uses the above data, and run the following strategy on it:
* Bid Size and Offer Size = 1000 USD notional (note, the actual traded base quantity on Binance BTCUSDT is BTC, so you will need to convert the Sizes to the equivalent BTC amount at every quote “refresh”
* Buy and Sell Margins = 3 basis points i.e. the distance between your bid or offer quote and the mid market in absolute terms.
* Maximum position before hedging = 5000 USD notional i.e. if your position exceeds this either on the buy or sell side, your strategy stops quoting on that side of the market and hedges your position. If your strategy uses a market order to exit the position, you will be rewarded one less point. A passive hedger approach will be rewarded full points upon completion.
* You may assume that the latencies in each step of the market maker is 100 milliseconds. A realistic random latency would be closer to the “real” result.
* You may assume that the cost of trading is zero i.e. there are no commissions
* You may assume that a trade match happens when the trading price “crosses” your bid or offer order, for simplicity.

The output of the trades generated from this strategy should reside in a different dataframe, for the next step. Completion upto this point in the Assignment Set would reward a maximum of 7 points.


In [7]:
# Conversion of 1BTC to USD
b = BtcConverter()
BTC_in_USD=b.get_latest_price('USD')

In [8]:
BTC_in_USD

40128.4467

***